In [1]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime
from create_sap_table import create_table_leanx as ctl

import values, helpers
import master_data, text_data, purchasing_doc_data

In [2]:
# # text tables
# for method in (
#     text_data.domain_fixed_values, 
#     text_data.sales_doc_types, 
#     text_data.sales_organizations,
#     text_data.distribution,
#     text_data.sales_doc_item_categories,
#     text_data.sales_doc_rejection_reasons,
#     text_data.system_status,
#     text_data.blocking_reasons,
#     text_data.releases,
#     text_data.purchasing_doc_types,
#     text_data.purchasing_organizations
# ):
#     table_dict = method()
#     for k, v in table_dict.items():
#         table_name = k.split('_')[0]
#         all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
#         df = pd.concat([all_cols, pd.DataFrame(v.values())])
#         df.to_csv(f'data/P2P/OCPM/text/{table_name}.csv', index=False)

In [3]:
# # master tables
# for method in (
#     master_data.users,
#     master_data.customers_and_vendors, 
#     master_data.plants, 
#     master_data.materials, 
#     master_data.material_support,
#     master_data.routes,
#     master_data.company_codes
# ):
#     table_dict = method()
#     for k, v in table_dict.items():
#         table_name = k.split('_')[0]
#         all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
#         df = pd.concat([all_cols, pd.DataFrame(v.values())])
#         df.to_csv(f'data/P2P/OCPM/master/{table_name}.csv', index=False)

In [4]:
def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

In [11]:
MARC = pd.read_csv('data/P2P/OCPM/master/MARC.csv')
MARA = pd.read_csv('data/P2P/OCPM/master/MARA.csv')
MAKT = pd.read_csv('data/P2P/OCPM/master/MAKT.csv')
LFB1 = pd.read_csv('data/P2P/OCPM/master/LFB1.csv')
LFA1 = pd.read_csv('data/P2P/OCPM/master/LFA1.csv')

all_prices = {}
all_no_contract_probabilities = {}

# get price and material group by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_no_contract_probabilities[nr] = attr['no_contract_probability']

def get_params():
    company_code = random.choice(list(values.om_company_codes.keys()))
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    konnr = f'{str(uuid.uuid4())[-15:]}'
    lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))

    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))
    quantities = [random.randint(12, 60)*12 for _ in range(len(matnrs))] # x dozens of everything
    
    prices = []
    no_contract_probabilities = []
    for i in range(len(matnrs)):
        prices.append(all_prices[matnrs[i]])
        no_contract_probabilities.append(all_no_contract_probabilities[matnrs[i]])
    
    item_has_contract = [True if random.random() < no_contract_probabilities[j] else False for j in range(len(no_contract_probabilities))]

    has_contract = True 
    for hc in item_has_contract:
        if hc == False:
            has_contract = False
            break
    
    has_pr_price_mismatch = False
    pr_prices = [p*(1+min(0.5,random.random())) for p in prices]

    requested_by = get_user_name(0.1)
    
    purchasing_org=random.choice(list(values.proc_purchasing_orgs.keys()))
    new_lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    payment_term = random.choice(list(LFB1[LFB1['LIFNR'] == lifnr]['ZTERM']))
    
    is_free_text = True if random.random() < 0.5 else False
    free_text_materials = matnrs # TODO change with material description


    params = {
        'matnrs': matnrs,
        'konnr': konnr,
        'lifnr': lifnr,
        'plant': plant,
        'quantities': quantities,
        'prices': prices, # considered contract and PO prices
        'company_code': company_code,
        'purchasing_org': purchasing_org,
        'payment_term': payment_term,
        'requested_by': requested_by,
        'item_has_contract': item_has_contract,
        'has_contract': has_contract,
        'has_pr_price_mismatch': False, # not applied in Contrace Usage use case - use if needed for Contract Leakage
        'pr_prices': [], # not applied in Contrace Usage use case - use if needed for Contract Leakage



        
        'is_free_text': is_free_text,
        'free_text_materials': free_text_materials,
        'new_payment_term': 'Z090',
        'new_vendor': new_lifnr,
    }

    return params

In [40]:
x = []
for i in range(1_000):
    x += get_params()['item_has_contract']

In [41]:
len(x)

14933

In [42]:
y = [i for i in x if i == True]

In [43]:
len(y)/len(x)

0.4206790330141298

In [13]:
purchasing_doc_tables = {
    'EBAN_json': {}, 
	'CDHDR_json': {},
	'CDPOS_json': {},
	'EKKO_json': {},
	'EKPO_json': {},
	'NAST_json': {},
	'MSEG_json': {},
	'EKBE_json': {},
	'RBKP_json': {},
	'RSEG_json': {},
	'EKET_json': {},
}
for i in range(1_000):
	"""
	0: Approve PO
	1: Create PO
	2: Send PO
	3: Post GR
	4: Create Vendor Invoice

	5: Change Payment Term
	6: Change Vendor
	7: Change Quantity
	"""

	# Create Contract
	# Create PR
	
	params = get_params()
	latest_date = helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2023, 9, 1))
	latest_time = helpers.generate_random_time()
	purchasing = purchasing_doc_data.Purchasing(params=params, start_date=latest_date, index=i)
	preq_creation_date = latest_date

	purchasing.create_contract( # TODO check if 1:1 mapping between contracts, PRs, and POs creates an issue
		aedat=latest_date,
		ernam=get_user_name(0.5)
	)

	purchasing.create_purchase_requisition_item(
		badat=preq_creation_date, 
		ernam=get_user_name(0.5),
	)
	latest_date += helpers.UPTO_WEEK()

	purchasing.approve_purchase_order(
		aedat=latest_date,
		ernam = get_user_name(0.5)
	)
	latest_date += helpers.UPTO_WEEK()

	purchasing.create_purchase_order(
		aedat=latest_date,
		ernam=get_user_name(0.5)
	)
	latest_date += helpers.UPTO_WEEK()

	purchasing.send_purchase_order(
		usnam=get_user_name(0.5),
		erdat=latest_date
	)
	latest_date += helpers.UPTO_WEEK()

	purchasing.post_goods_receipt(
		cpudt=latest_date,
		usnam=get_user_name(0.5),
		atime=latest_time
	)
	latest_date += helpers.UPTO_WEEK()

	purchasing.create_vendor_invoice(
		cupdt=latest_date,
		ernam=get_user_name(0.5),
	)
	latest_date += helpers.UPTO_WEEK()

	purchasing.change_payment_term(
		udate=latest_date,
		ernam=get_user_name(0.5)
	)
	latest_date += helpers.UPTO_WEEK()

	purchasing.change_vendor(
		udate=latest_date,
		ernam=get_user_name(0.5)
	)
	latest_date += helpers.UPTO_WEEK()

	new_quantity_lines = [0]
	new_quanity_quantities = [99]
	purchasing.change_quantity(
		badat=latest_date,
		ernam=get_user_name(0.5),
		line_numbers=new_quantity_lines,
		line_quantities=new_quanity_quantities
	)


	for k, v in purchasing.tables.items():
			for entry_key in list(v.keys()):
				purchasing_doc_tables[k][entry_key] = purchasing.tables[k][entry_key]

In [6]:
for table, rows in purchasing_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    df = pd.concat([all_cols, pd.DataFrame(rows.values())])
    df.to_csv(f'data/P2P/OCPM/purchasing-document/{table_name}.csv', index=False)